In [ ]:
%matplotlib inline

# Iris/dask dataset processing investigation

## Introduction

This demos using dask functionality beyond the `array` module to help with Iris processing. Specifically, in this notebook we will demo alternative approaches for loading numerous and/or large datasets into Iris.

Three approaches will be compared:

* The standard Iris load
* ~~Wrapping Iris load calls in a **dask bag** generated from a sequence (this is slow so will not be investigated further)~~
* Wrapping Iris load calls in a **dask bag** generated from a **delayed** call

These options will be compared with two simple metrics:

- Ease of use
- Runtime

## Setup

Below are the functions used to load the dataset. There is one function for each of the standard Iris load and the bag generated from a sequence. The bag generated from a delayed call requires two functions; one which is delayed, one to call the delayed function.

### Imports

In [ ]:
import os
import time

import dask
import dask.array as da
import dask.bag as db
import dask.delayed as delayed
import iris

### Dask processing options

Define options on how dask is to process computation of graphs. Choose one of these!

In [ ]:
from distributed import Client

host_subnet = 55
s = '10.154.1.{}:8776'.format(host_subnet)

client = Client(s)

In [ ]:
print dask.context._globals

### Loader function

In [ ]:
def delay_load(fp, seq):
    dlyds = [delayed(iris.load)(os.path.join(fp, pattern)) for pattern in seq]
    cs = db.from_delayed(dlyds)
    return iris.cube.CubeList(cs.compute())

## Test!

Run each loader on some sample data and print the output.

Using **sample PP data** at `/project/euro4_hindcast/WIND-ATLAS_EURO4-RERUN/2015/06/18Z`:

In [ ]:
fp = '/project/euro4_hindcast/WIND-ATLAS_EURO4-RERUN/2015/06/18Z'
fn = '*.pp'
seq = os.listdir(fp)

In [ ]:
cubes = delay_load(fp, seq)

In [ ]:
print cubes

In [ ]:
cubes[0].core_data()

### Data processing

The functions required to apply some post-processing to the loaded datasets. Two bits of processing are performed:

* `x` and `y` wind are converted to wind speed and direction, and
* the variance of wind speed and direction across model levels is calculated.

In [ ]:
def xy_to_wspd_and_dir(x_cube, y_cube):
    """
    Post-processing, part 1: mathematics.
    Converting x and y wind to speed and direction.

    """
    wspd_data = (x_cube.core_data()**2 + y_cube.core_data()**2) ** 0.5
    wspd_cube = x_cube.copy(data=wspd_data)
    wspd_cube.rename('wind_speed')
    wspd_cube.units = 'm s-1'

    theta_data = da.arctan(x_cube.core_data() / y_cube.core_data())
    theta_cube = y_cube.copy(data=theta_data)
    theta_cube.rename('wind_from_direction')
    theta_cube.units = 'degrees'

    return wspd_cube, theta_cube


def mln_variance(wspd_cube, wdir_cube):
    """
    Post-processing, part 2: statistical analysis.
    Calculate the variance in wind speed and direction over model levels.

    """
    wspd_var_cube = wspd_cube.collapsed('model_level_number',
                                        iris.analysis.VARIANCE)
    wdir_var_cube = wdir_cube.collapsed('model_level_number',
                                        iris.analysis.VARIANCE)
    return wspd_var_cube, wdir_var_cube

Run the processing...

In [ ]:
# The x- and y-wind cubes are on different domains. This notwithstanding,
# the x-wind cube also has one more latitude point than the y-wind cube,
# which we arbitrarily chop off.
x_wind_cube = cubes[0][..., :-1, :]
y_wind_cube = cubes[1]

wspd_cube, theta_cube = xy_to_wspd_and_dir(x_wind_cube, y_wind_cube)
wspd_var_cube, wdir_var_cube = mln_variance(wspd_cube, theta_cube)

In [ ]:
print wspd_var_cube

In [ ]:
print wdir_var_cube

### Plot

Use holoviews to create an interactive plot across timesteps for wind speed and direction data.

In [ ]:
import geoviews as gv
import geoviews.feature as gf
import holoviews as hv

hv.notebook_extension()
%output holomap='auto'

In [ ]:
cl = gf.coastline(style=dict(edgecolor='w'))

In [ ]:
def slice_image(index):
    result = gv.Dataset(wspd_var_cube[index],
                        kdims=['grid_longitude', 'grid_latitude'], vdims=['wind_speed']
                       ).to(gv.Image)
    return result

In [ ]:
slice_image(5)

In [ ]:
dim_time = hv.Dimension('time',
                        values=range(31))

In [ ]:
%%output size=275
dmap = hv.DynamicMap(slice_image, kdims=[dim_time]) * cl
dmap

In [ ]:
def slice_image_realize(index):
    cube = wspd_var_cube[index]
    cube.data
    result = gv.Dataset(cube,
                        kdims=['grid_longitude', 'grid_latitude'], vdims=['wind_speed']
                       ).to(gv.Image)
    return result

In [ ]:
%%output size=275
dmap = hv.DynamicMap(slice_image_realize, kdims=[dim_time]) * cl
dmap

### Compute results

In [ ]:
hv.HoloMap(dmap[set(range(31))])

In [ ]:
wspd_var_data = wspd_var_cube.data

In [ ]:
wdir_var_data = wdir_var_cube.data

In [ ]:
result = gv.Dataset(wspd_var_cube,
                    kdims=['time', 'grid_longitude', 'grid_latitude'], vdims=['wind_speed']
                   )
result

In [ ]:
result.data